# Import packages

In [1]:
import os
import json
import cv2
import time
import datetime
import shutil   # ONLY USED FOR CLEARING EXISTING FRAMES
from common_functionality import *
from deepface import DeepFace
import face_recognition, face_rec
import numpy as np

# Frame Extraction

In [2]:
def create_json_file(file_path):
    with open(file_path, 'w') as f:
        f.write(json.dumps({}))

In [32]:
def reset_frames_list(file_path):
    with open(file_path, 'r+') as f:
        processed_data = json.load(f)
        if 'frames_list' in processed_data.keys():
            processed_data['frames_list'].clear()
            print(processed_data)
        
        f.seek(0)
        json.dump(processed_data, f, indent=4)
        f.truncate()

In [33]:
def update_json_content(file_path, pic_dict):
    with open(file_path, 'r+') as f:
        processed_data = json.load(f)
        if 'frames_list' not in processed_data.keys():
            processed_data['frames_list'] = []
        
        processed_data['frames_list'].append(pic_dict)
        f.seek(0)
        json.dump(processed_data, f, indent=4)
        f.truncate()

In [12]:
videos_dir = 'videos_db'
imgs_dir = 'vid_frames'
data_folder = 'data'
video_data_folder = 'video_data'
full_vid_data_path = os.path.join(data_folder, video_data_folder)

In [16]:
#NOTE: ONLY TO BE USED TO PROCESS THE VIDEOS DONE BY MARK LAWRENCE
videos_dir = 'videos_db_ml'
imgs_dir = 'vid_frames_ml'
data_folder = 'data'
video_data_folder = 'video_data_ml'
full_vid_data_path = os.path.join(data_folder, video_data_folder)

In [17]:
if not os.path.exists(full_vid_data_path):
    os.mkdir(full_vid_data_path)

In [18]:
frames_subDirs = []

print('List of available of videos with extracted frames:')
for root, dirs, files in os.walk(videos_dir, topdown=True):
    for vid in files:
        if vid.endswith('.mp4'):
            frames_subDirs.append(vid)

videos_sorted = sorted(frames_subDirs, key=lambda x: int(x.split('_')[-1].split('.')[0]))

print(videos_sorted)

List of available of videos with extracted frames:
['video_1.mp4', 'video_2.mp4']


In [15]:
seconds_between_skips = 10

for video_name in videos_sorted:
    if video_name.endswith('mp4'):
        video_data_path = os.path.join(full_vid_data_path, f'{os.path.splitext(video_name)[0]}.json')
        if not os.path.exists(video_data_path):
            create_json_file(video_data_path)
        else:
            reset_frames_list(video_data_path)
        video_path = f'{videos_dir}/{video_name}'
        #_, video_name = video_path.split('/')
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        duration_in_seconds = total_frames // fps
        #print(duration_in_seconds)
        #print(fps)

        frame_interval = int(fps * seconds_between_skips)

        video_imgs_path = f'{imgs_dir}/{video_name[:-4]}'

        if not os.path.exists(video_imgs_path):
            os.makedirs(video_imgs_path)
        else:
            if len(os.listdir(video_imgs_path)) > 0:
                clear_folder(video_imgs_path)

        print("Processing "+str(video_name))
        start = time.time()

        start_timestamp_seconds = 75
        frame_count = 1

        # Set the position in milliseconds to the next timestamp
        cap.set(cv2.CAP_PROP_POS_MSEC, start_timestamp_seconds * 1000)
        new_ts_dt = datetime.timedelta(seconds=start_timestamp_seconds)
        while True:
            
            ret, frame = cap.read()
            if not ret:
                break
            else:
                # Save the frame as an image
                saved_frame_count = frame_count-1
                new_timestamp = (start_timestamp_seconds + (seconds_between_skips * frame_count))
                frame_filename = os.path.join(video_imgs_path, f"frame_{saved_frame_count:04d}.png")
                cv2.imwrite(frame_filename, frame)
                print(f"Exported frame {frame_count} at {new_ts_dt}")

                # Write frame number and timestamp to JSON file
                frame_dict = {'frame_num': f"frame_{saved_frame_count:04d}", 'frame_timestamp': str(new_ts_dt)}
                #print(frame_dict)
                update_json_content(video_data_path, frame_dict)

                # Update timestamp
                new_ts_dt = datetime.timedelta(seconds=new_timestamp)
                #print(f"Exported frame {frame_count} at {new_timestamp // (60*60):02d}:{new_timestamp // 60:02d}:{new_timestamp % 60:02d}")

                # Skip the desired number of frames
                cap.set(cv2.CAP_PROP_POS_MSEC, new_timestamp * 1000)
                frame_count += 1


        end = time.time()
        print("Video "+str(video_name)+" has been processed!")

        seconds = end - start
        print (f"Time taken : {seconds} seconds")
        print("----------------------------------------------------")

        cap.release()
    #break

cv2.destroyAllWindows()

print(f"All frames extracted from the videos in the directory \"{videos_dir}\" can be found in the directory \"{imgs_dir}\"")


Folder cleared
Processing video_2.mp4
Exported frame 1 at 0:01:15
Exported frame 2 at 0:01:25
Exported frame 3 at 0:01:35
Exported frame 4 at 0:01:45
Exported frame 5 at 0:01:55
Exported frame 6 at 0:02:05
Exported frame 7 at 0:02:15
Exported frame 8 at 0:02:25
Exported frame 9 at 0:02:35
Exported frame 10 at 0:02:45
Exported frame 11 at 0:02:55
Exported frame 12 at 0:03:05
Exported frame 13 at 0:03:15
Exported frame 14 at 0:03:25
Exported frame 15 at 0:03:35
Exported frame 16 at 0:03:45
Exported frame 17 at 0:03:55
Exported frame 18 at 0:04:05
Exported frame 19 at 0:04:15
Exported frame 20 at 0:04:25
Exported frame 21 at 0:04:35
Exported frame 22 at 0:04:45
Exported frame 23 at 0:04:55
Exported frame 24 at 0:05:05
Exported frame 25 at 0:05:15
Exported frame 26 at 0:05:25
Exported frame 27 at 0:05:35
Exported frame 28 at 0:05:45
Exported frame 29 at 0:05:55
Exported frame 30 at 0:06:05
Exported frame 31 at 0:06:15
Exported frame 32 at 0:06:25
Exported frame 33 at 0:06:35
Exported frame

In [35]:
#NOTE: ONLY TO BE USED TO PROCESS THE VIDEOS DONE BY MARK LAWRENCE
seconds_between_skips = 10

for video_name in videos_sorted:
    if video_name.endswith('mp4'):
        video_data_path = os.path.join(full_vid_data_path, f'{os.path.splitext(video_name)[0]}.json')
        if not os.path.exists(video_data_path):
            create_json_file(video_data_path)
        else:
            reset_frames_list(video_data_path)
        video_path = f'{videos_dir}/{video_name}'
        #_, video_name = video_path.split('/')
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        duration_in_seconds = total_frames // fps
        #print(duration_in_seconds)
        #print(fps)

        frame_interval = int(fps * seconds_between_skips)

        video_imgs_path = f'{imgs_dir}/{video_name[:-4]}'

        if not os.path.exists(video_imgs_path):
            os.makedirs(video_imgs_path)
        else:
            if len(os.listdir(video_imgs_path)) > 0:
                clear_folder(video_imgs_path)

        print("Processing "+str(video_name))
        start = time.time()

        start_timestamp_seconds = 75
        frame_count = 1

        # Set the position in milliseconds to the next timestamp
        cap.set(cv2.CAP_PROP_POS_MSEC, start_timestamp_seconds * 1000)
        new_ts_dt = datetime.timedelta(seconds=start_timestamp_seconds)
        while True:
            
            ret, frame = cap.read()
            if not ret:
                break
            else:
                # Save the frame as an image
                saved_frame_count = frame_count-1
                new_timestamp = (start_timestamp_seconds + (seconds_between_skips * frame_count))
                frame_filename = os.path.join(video_imgs_path, f"frame_{saved_frame_count:04d}.png")
                cv2.imwrite(frame_filename, frame)
                print(f"Exported frame {frame_count} at {new_ts_dt}")

                # Write frame number and timestamp to JSON file
                frame_dict = {'frame_num': f"frame_{saved_frame_count:04d}", 'frame_timestamp': str(new_ts_dt)}
                #print(frame_dict)
                update_json_content(video_data_path, frame_dict)

                # Update timestamp
                new_ts_dt = datetime.timedelta(seconds=new_timestamp)
                #print(f"Exported frame {frame_count} at {new_timestamp // (60*60):02d}:{new_timestamp // 60:02d}:{new_timestamp % 60:02d}")

                # Skip the desired number of frames
                cap.set(cv2.CAP_PROP_POS_MSEC, new_timestamp * 1000)
                frame_count += 1


        end = time.time()
        print("Video "+str(video_name)+" has been processed!")

        seconds = end - start
        print (f"Time taken : {seconds} seconds")
        print("----------------------------------------------------")

        cap.release()
    #break

cv2.destroyAllWindows()

print(f"All frames extracted from the videos in the directory \"{videos_dir}\" can be found in the directory \"{imgs_dir}\"")


{'video_date': '26-02-2025', 'frames_list': []}
Folder cleared
Processing video_1.mp4
Exported frame 1 at 0:01:15
Exported frame 2 at 0:01:25
Exported frame 3 at 0:01:35
Exported frame 4 at 0:01:45
Exported frame 5 at 0:01:55
Exported frame 6 at 0:02:05
Exported frame 7 at 0:02:15
Exported frame 8 at 0:02:25
Exported frame 9 at 0:02:35
Exported frame 10 at 0:02:45
Exported frame 11 at 0:02:55
Exported frame 12 at 0:03:05
Exported frame 13 at 0:03:15
Exported frame 14 at 0:03:25
Exported frame 15 at 0:03:35
Exported frame 16 at 0:03:45
Exported frame 17 at 0:03:55
Exported frame 18 at 0:04:05
Exported frame 19 at 0:04:15
Exported frame 20 at 0:04:25
Exported frame 21 at 0:04:35
Exported frame 22 at 0:04:45
Exported frame 23 at 0:04:55
Exported frame 24 at 0:05:05
Exported frame 25 at 0:05:15
Exported frame 26 at 0:05:25
Exported frame 27 at 0:05:35
Exported frame 28 at 0:05:45
Exported frame 29 at 0:05:55
Exported frame 30 at 0:06:05
Exported frame 31 at 0:06:15
Exported frame 32 at 0:0

# Facial ROI Extraction

## JSON functionality

<b>NOTE: Data folder must contain video data before running below two functions</b>

In [2]:
# Function to add number of extracted facial ROIs to video data file
def add_num_rois_to_json(file_path, base_img_name, num_rois):
    with open(file_path, 'r+') as f:
        processed_data = json.load(f)
        if 'frames_list' not in processed_data.keys():
            processed_data['frames_list'] = []
        
        for index, frame in enumerate(processed_data['frames_list']):
            if frame['frame_num'] == base_img_name:
                frame_dict = frame.copy()
                pos = index
                break
        
        frame_dict['num_of_rois'] = num_rois

        processed_data['frames_list'][pos] = frame_dict
        f.seek(0)
        json.dump(processed_data, f, indent=4)
        f.truncate()  # This prevents leftover characters from previous write

# Function to add extracted facial ROI names to video data file
def add_roi_names_to_json(file_path, base_img_name, rois_list):
    with open(file_path, 'r+') as f:
        processed_data = json.load(f)
        if 'frames_list' not in processed_data.keys():
            processed_data['frames_list'] = []
        
        for index, frame in enumerate(processed_data['frames_list']):
            if frame['frame_num'] == base_img_name:
                frame_dict = frame.copy()
                pos = index
                break
        
        if len(rois_list) > 0:
            frame_dict['rois_list'] = [{'roi_num': roi_name} for roi_name in rois_list]
        else:
            frame_dict['rois_list'] = []

        processed_data['frames_list'][pos] = frame_dict
        f.seek(0)
        json.dump(processed_data, f, indent=4)
        f.truncate()  # This prevents leftover characters from previous write

def add_encodings_to_json(file_path, base_img_name, encodings_list):
    with open(file_path, 'r+') as f:
        processed_data = json.load(f)
        if 'frames_list' not in processed_data:
            processed_data['frames_list'] = []

        frame_dict = None
        for index, frame in enumerate(processed_data['frames_list']):
            if frame['frame_num'] == base_img_name:
                frame_dict = frame
                pos = index
                break

        if frame_dict is None:
            raise Exception(f"Frame {base_img_name} not found in {file_path}")

        # Match encodings to ROI entries
        rois_list = frame_dict.get('rois_list', [])
        for i, roi in enumerate(rois_list):
            if i < len(encodings_list):
                facial_encoding = encodings_list[i].get('facial_area', {})
                roi['facial_encoding'] = {k: v for k, v in facial_encoding.items() if not (k == 'left_eye' or k == 'right_eye')}
            #else:
                #roi['facial_encoding'] = []  # or None if you prefer
        
        processed_data['frames_list'][pos] = frame_dict
        f.seek(0)
        json.dump(processed_data, f, indent=4)
        f.truncate()

def check_for_intermission_frame(file_path, base_img_name, intermission_path):
    intermission_found = False
    with open(file_path, 'r+') as f:
        processed_data = json.load(f)
        if 'frames_list' not in processed_data.keys():
            processed_data['frames_list'] = []
        
        for index, frame in enumerate(processed_data['frames_list']):
            if frame['frame_num'] == base_img_name:
                frame_dict = frame.copy()
                pos = index
                break
    
    with open(intermission_path, 'r+') as f:
        intermission_data = json.load(f)
    
    print(processed_data['frames_list'][pos]['frame_timestamp'])
    timestamp_h, timestamp_m, timestamp_s = map(int, processed_data['frames_list'][pos]['frame_timestamp'].split(':'))
    timestamp_delta = datetime.timedelta(hours=timestamp_h, minutes=timestamp_m, seconds=timestamp_s)
    
    for intermission in intermission_data['commercial_times']:
        start = intermission['start_time']
        start_h, start_m, start_s = map(int, start.split(':'))
        start_delta = datetime.timedelta(hours=start_h, minutes=start_m, seconds=start_s)
        end = intermission['end_time']
        end_h, end_m, end_s = map(int, end.split(':'))
        end_delta = datetime.timedelta(hours=end_h, minutes=end_m, seconds=end_s)
        if timestamp_delta > start_delta and timestamp_delta <= end_delta:
            intermission_found = True
            break
            #return "Cannot perform facial ROI extraction"
    
    return intermission_found
        #print("Cannot perform facial ROI extraction" if timestamp_delta > start_delta and timestamp_delta <= end_delta else "Can perform facial ROI extraction")      

## Parameter declaration and initialisation

In [20]:
padding = 25

frames_dir = 'vid_frames'
frames_subDirs = []
rois_dir = 'face_rois_test_dt_2'
data_folder = 'data'
video_data_path = os.path.join(data_folder, 'video_data')
intermission_data_path = os.path.join('data', 'intermission_data')

# This is essential to avoid any unnecessary extraction of facial ROIs from ads at the bottom of the video
y_crop = 81

In [30]:
vid_frames_folders = [folder for folder in os.listdir(".") if os.path.isdir(os.path.join(".", folder)) and folder.startswith("vid_frames")]

for index, frames_dir in enumerate(vid_frames_folders):
    print(f'{index+1}: {frames_dir}')

1: test_vid_frames
2: test_vid_frames_ml


In [38]:
valid_choice = False
while not valid_choice:
    index_choice = input('Enter index of video frames directory: ')

    if index_choice.isnumeric():
        if int(index_choice) > len(vid_frames_folders) or int(index_choice) <= 0:
            valid_choice = False
            print('Invalid choice!')
        else:
            valid_choice = True
            frames_dir = vid_frames_folders[int(index_choice)-1]
            rois_dir = f'face_rois_dt_{"_".join(frames_dir.split("_")[-1:])}' if frames_dir != "vid_frames" else "face_rois_dt"
            video_data_path = os.path.join(data_folder, f'video_data_{"_".join(frames_dir.split("_")[-1:])}') if frames_dir != "vid_frames" else os.path.join(data_folder, 'video_data')
            print("Video Frames Folder selected:", frames_dir)
            print(rois_dir)
            print(video_data_path)

    else:
        valid_choice = False
        print("Index entered is not numeric!")

Video Frames Folder selected: test_vid_frames_ml
test_face_rois_dt_ml
test_data\video_data_ml


In [39]:
if not os.path.exists(rois_dir):
    os.mkdir(rois_dir)

## Facial ROI execution

### Asking user for directory of video to process

In [40]:
frames_subDirs = []

print('List of available of videos with extracted frames:')
for root, dirs, files in os.walk(frames_dir, topdown=True):
    for subDir in dirs:
        frames_subDirs.append(subDir)

frames_subDirs_sorted = sorted(frames_subDirs, key=lambda x: int(x.split('_')[-1]))

for index, subDir in enumerate(frames_subDirs_sorted):
    print(f'{index+1}: {os.path.join(frames_dir, subDir)}')

List of available of videos with extracted frames:
1: test_vid_frames_ml\video_1
2: test_vid_frames_ml\video_2


In [44]:
valid_choice = False
dir_chosen = ""
dir_chosen_full_path = ""
while not valid_choice:
    index_choice = input('Enter index of directory that you want to retrieve face ROIs: ')

    if index_choice.isnumeric():
        if int(index_choice) > len(frames_subDirs_sorted) or int(index_choice) <= 0:
            valid_choice = False
            print('Invalid choice!')
        else:
            valid_choice = True
            dir_chosen = frames_subDirs_sorted[int(index_choice)-1]
            dir_chosen_full_path = os.path.join(frames_dir, dir_chosen)
            chosen_video_data_path = os.path.join(video_data_path, f'{dir_chosen}.json')
            chosen_video_intermission_path = os.path.join(intermission_data_path, f'{dir_chosen}.json') if "_ml" not in frames_dir else os.path.join(f'{intermission_data_path}_ml', f'{dir_chosen}.json')
            print("Directory selected:", dir_chosen_full_path)
            print("Video selected:", dir_chosen)
    else:
        valid_choice = False
        print("Index entered is not numeric!")

Directory selected: test_vid_frames_ml\video_1
Video selected: video_1


In [45]:
rois_subDir = os.path.join(rois_dir, dir_chosen)

if not os.path.exists(rois_subDir):
    os.mkdir(rois_subDir)
else:
    if len(os.listdir(rois_subDir)) > 0:
        clear_folder(rois_subDir)

## Validate if video data exists

In [46]:
# Check if data file for selected video exists in the data folder
video_data_file_exists = check_if_data_file_exists(dir_chosen)

if not video_data_file_exists:
    raise Exception(f'Data file for {dir_chosen} does not exist in {video_data_path}!')
    exit(0)

# Check if there is a list of extracted frames in the selected video data file
video_data_exists = validate_data_file(chosen_video_data_path)

print(chosen_video_data_path)
print(chosen_video_intermission_path)

if not video_data_exists:
    raise Exception(f'Please ensure that all frames from {dir_chosen} have been extracted before running this!')
    exit(0)

print("Chosen video data file is valid!")

test_data\video_data_ml\video_1.json
data\intermission_data_ml\video_1.json
Chosen video data file is valid!


## Extract facial ROIs from video frames

In [47]:
min_w, min_h = 20, 30

In [48]:
total_num_rois_extracted = 0
if video_data_exists:
    # Loop through each file in the directory
    for snapshot_img in os.listdir(dir_chosen_full_path):
        roi_names = []
        valid_encodings = []

        # Only take into account image files (which end in .jpg or .png)
        if snapshot_img.endswith('.jpg') or snapshot_img.endswith('.png'):
            snapshot_path = os.path.join(dir_chosen_full_path, snapshot_img)
            snapshot_basename = os.path.splitext(os.path.basename(snapshot_img))[0]
            snapshot_ext = os.path.splitext(os.path.basename(snapshot_img))[1]
            #print(snapshot_basename)
            #print(snapshot_ext)

            is_intermission_frame = check_for_intermission_frame(chosen_video_data_path, snapshot_basename, chosen_video_intermission_path)

            if is_intermission_frame:
                print(f'Intermission frame detected at {snapshot_basename}! Cannot extract potential face/s!')
                add_num_rois_to_json(chosen_video_data_path, os.path.splitext(snapshot_img)[0], 0)
                add_roi_names_to_json(chosen_video_data_path, os.path.splitext(snapshot_img)[0], [])
                continue

            # Load and crop image to avoid extracting faces from any ads from the bottom of the screen
            snapshot = face_rec.load_image_file(snapshot_path)
            snapshot = cv2.cvtColor(snapshot, cv2.COLOR_BGR2RGB)
            snapshot_crop = snapshot[:snapshot.shape[0]-y_crop,:]

            # Extract facial encodings from the processed image
            # If no faces are found, then program moves on to next image
            try:
                snapshot_encodings = DeepFace.represent(snapshot_crop, detector_backend='retinaface')
            except:
                print(f'No face detected in {snapshot_basename}!')
                print("------------------------------------------------------------------------------")
                add_num_rois_to_json(chosen_video_data_path, os.path.splitext(snapshot_img)[0], 0)
                add_roi_names_to_json(chosen_video_data_path, os.path.splitext(snapshot_img)[0], [])
                continue
            
            num_rois = len(snapshot_encodings)

            # Do not perform ROI extraction on frames with more than 2 ROIs (e.g. in commercials)
            if num_rois > 2:
                add_num_rois_to_json(chosen_video_data_path, os.path.splitext(snapshot_img)[0], 0)
                add_roi_names_to_json(chosen_video_data_path, os.path.splitext(snapshot_img)[0], [])
                continue
            
            # Extract facial ROIs using information generated from the encodings
            for index, encoding in enumerate(snapshot_encodings):
                #print(snapshot_path.split('/')[-1], '-',encoding['facial_area'])
                roi_x = encoding['facial_area']['x']
                roi_y = encoding['facial_area']['y']
                roi_w = encoding['facial_area']['w']
                roi_h = encoding['facial_area']['h']
                roi_left, roi_right = roi_x - padding, roi_x + roi_w + padding
                roi_top, roi_bottom = roi_y - padding, roi_y + roi_h + padding

                # Ensure dimensions are greater than 0 to avoid any errors
                if roi_top < 0:
                    roi_top = 0
                
                if roi_left < 0:
                    roi_left = 0

                roi_img = snapshot[roi_top:roi_bottom, roi_left:roi_right]
                #print(roi_img.shape)

                img_w, img_h = roi_img.shape[1], roi_img.shape[0]

                # Do not save any facial ROIs that are too small (e.g. in commercials)
                if roi_w < min_w or roi_h < min_h:
                    num_rois -= 1
                    continue
                
                # Declare and initalise the ROI filename and the location of the file in which it is to be saved
                roi_filename = f'{snapshot_basename}_{index:02d}{snapshot_ext}'
                roi_full_path = os.path.join(rois_subDir, roi_filename)

                # Save extracted ROI
                cv2.imwrite(roi_full_path, roi_img)
                roi_names.append(os.path.splitext(roi_filename)[0])
                valid_encodings.append(encoding)
            
            # Inform user about progress
            print(f"Extracted {num_rois} facial ROIs from {snapshot_basename}: {roi_names}")
            print("------------------------------------------------------------------------------")

            total_num_rois_extracted += num_rois
            
            # Write information to video data file
            add_num_rois_to_json(chosen_video_data_path, os.path.splitext(snapshot_img)[0], num_rois)
            add_roi_names_to_json(chosen_video_data_path, os.path.splitext(snapshot_img)[0], roi_names)
            add_encodings_to_json(chosen_video_data_path, os.path.splitext(snapshot_img)[0], valid_encodings)

    # Inform user that facial ROI extraction process is complete
    print(f"A total of {total_num_rois_extracted} facial ROIs for {dir_chosen} have been saved in {rois_subDir}")
else:
    raise Exception(f'Please ensure that all frames from {dir_chosen} have been extracted before running this!')

0:01:15
Intermission frame detected at frame_0000! Cannot extract potential face/s!
0:01:25
Extracted 1 facial ROIs from frame_0001: ['frame_0001_00']
------------------------------------------------------------------------------
0:01:35
Extracted 1 facial ROIs from frame_0002: ['frame_0002_00']
------------------------------------------------------------------------------
0:01:45
Extracted 2 facial ROIs from frame_0003: ['frame_0003_00', 'frame_0003_01']
------------------------------------------------------------------------------
0:01:55
Extracted 2 facial ROIs from frame_0004: ['frame_0004_00', 'frame_0004_01']
------------------------------------------------------------------------------
0:02:05
Extracted 1 facial ROIs from frame_0005: ['frame_0005_00']
------------------------------------------------------------------------------
0:02:15
Extracted 1 facial ROIs from frame_0006: ['frame_0006_00']
------------------------------------------------------------------------------
0:02:2